In [1]:
!pip install supervision
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from ultralytics import YOLO
import cv2
import numpy as np
import json

# ============================
# 🚀 Khởi tạo mô hình và video
# ============================

# Tải mô hình YOLO đã huấn luyện
model_pitch = YOLO("/content/drive/MyDrive/Thử nghiệm tối ưu/best_n.pt")

# Mở video đầu vào
video_path = "/content/drive/MyDrive/Thử nghiệm tối ưu/Test30s.mp4"
cap = cv2.VideoCapture(video_path)

# Thiết lập video đầu ra
save_output = True
if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter("output_detection_optical_flow.mp4", fourcc, fps, (width, height))

# ============================
# 🔧 Helper functions
# ============================

def calculate_iou(box1, box2):
    """
    Tính toán Intersection over Union (IoU) giữa 2 bounding box.

    Args:
        box1 (list/tuple): [x1, y1, x2, y2] của box thứ nhất.
        box2 (list/tuple): [x1, y1, x2, y2] của box thứ hai.

    Returns:
        float: Giá trị IoU (0.0 → 1.0).
    """
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Tính toán diện tích giao nhau
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    inter_area = max(0, x2_i - x1_i) * max(0, y2_i - y1_i)

    # Diện tích từng box
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)

    # Diện tích hợp
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area > 0 else 0.0


def apply_nms(boxes, confs, labels, iou_threshold=0.7, conf_threshold=0.1):
    """
    Áp dụng Non-Maximum Suppression (NMS) để loại bỏ box trùng lặp.

    Args:
        boxes (list): Danh sách box dạng [x1, y1, x2, y2, conf].
        confs (list): Danh sách confidence score.
        labels (list): Danh sách nhãn.
        iou_threshold (float): Ngưỡng IoU để loại bỏ box trùng.
        conf_threshold (float): Ngưỡng confidence để giữ box.

    Returns:
        tuple: (new_boxes, new_confs, new_labels) sau khi NMS.
    """
    if not boxes:
        return [], [], []

    indices = cv2.dnn.NMSBoxes(
        [box[:4] for box in boxes],
        confs,
        conf_threshold,
        iou_threshold
    )

    if len(indices) > 0:
        indices = indices.flatten().tolist()
        new_boxes = [boxes[i] for i in indices]
        new_confs = [confs[i] for i in indices]
        new_labels = [labels[i] for i in indices]
        return new_boxes, new_confs, new_labels

    return [], [], []


def match_boxes_by_iou(boxes1, boxes2, iou_threshold=0.7):
    """
    Ghép cặp box giữa 2 tập hợp dựa trên IoU.

    Args:
        boxes1 (list): Danh sách box đầu tiên [x1, y1, x2, y2, conf].
        boxes2 (list): Danh sách box thứ hai [x1, y1, x2, y2, conf].
        iou_threshold (float): Ngưỡng IoU để coi là match.

    Returns:
        list: Danh sách tuple (i, j) với i là index trong boxes1, j là index trong boxes2.
    """
    matches = []
    used = set()

    for i, b1 in enumerate(boxes1):
        best_iou = 0.0
        best_j = -1
        for j, b2 in enumerate(boxes2):
            if j in used:
                continue
            iou = calculate_iou(b1[:4], b2[:4])
            if iou > best_iou and iou >= iou_threshold:
                best_iou = iou
                best_j = j
        if best_j != -1:
            matches.append((i, best_j))
            used.add(best_j)

    return matches

def draw_detections(frame, boxes_with_conf, labels, keypoints=None):
    """
    Vẽ bounding box, nhãn và keypoints lên khung hình.

    Args:
        frame (np.ndarray): Khung hình gốc.
        boxes_with_conf (list): Danh sách box [x1, y1, x2, y2, conf].
        labels (list): Danh sách nhãn tương ứng.
        keypoints (list, optional): Danh sách tọa độ keypoints.

    Returns:
        np.ndarray: Khung hình đã vẽ.
    """
    color_map = {'pitch': (0, 255, 0)}

    # Vẽ bounding box
    for box, label in zip(boxes_with_conf, labels):
        if len(box) >= 5:
            x1, y1, x2, y2, conf_val = map(float, box[:5])
        else:
            x1, y1, x2, y2 = map(float, box[:4])
            conf_val = 0.0
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        color = color_map.get(label, (200, 200, 200))
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {conf_val:.2f}", (x1, max(y1 - 10, 15)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Vẽ keypoints
    if keypoints:
        for idx, (x, y) in enumerate(keypoints):
            if x > 0 and y > 0:
                x, y = map(int, [x, y])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                cv2.putText(frame, str(idx + 1), (x + 6, y - 6),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return frame

# ============================
# 🔍 Dự đoán & lưu JSON
# ============================
full_predictions = []
cap = cv2.VideoCapture(video_path)
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

     #Chạy YOLO cho khung hình
    results_pitch = model_pitch(frame)[0]

    cur_boxes, cur_labels, cur_confs = [], [], []

    # Thu thập box cho nhãn 'pitch'
    for b in results_pitch.boxes:
        box = [float(x) for x in b.xyxy[0].cpu().numpy()]
        conf_val = float(b.conf[0])
        label = results_pitch.names[int(b.cls[0])]
        if label == "pitch":  # chỉ giữ nhãn pitch
            cur_boxes.append(box + [conf_val])
            cur_labels.append(label)
            cur_confs.append(conf_val)

    # Thu thập keypoints nếu có
    cur_kps = []
    if hasattr(results_pitch, "keypoints") and results_pitch.keypoints is not None and len(results_pitch.keypoints.xy) > 0:
        cur_kps = [[float(x), float(y)] for x, y in results_pitch.keypoints.xy[0].cpu().numpy()]

   # Lưu dữ liệu khung hình
frame_data = {
    "frame_id": frame_idx,
    "pitch": {
        "boxes": [],
        "keypoints": cur_kps
    }  # đã bỏ "players" theo yêu cầu
}

# Chỉ thêm box có nhãn 'pitch'
for box, conf, label in zip(cur_boxes, cur_confs, cur_labels):
    if label == "pitch":
        box_data = {"bbox": box[:4], "conf": conf, "class": label}
        frame_data["pitch"]["boxes"].append(box_data)

full_predictions.append(frame_data)
frame_idx += 1

# Lưu file JSON
json_file = "predictions_full.json"
with open(json_file, "w") as f:
    json.dump(full_predictions, f, indent=4)

cap.release()
print(f"✅ Đã xử lý xong {frame_idx} khung hình. Kết quả lưu tại '{json_file}'.")

# ==== Tạo predictions_sparse.json với nội suy quang lưu ====
sparse_predictions = []
cap = cv2.VideoCapture(video_path)
frame_idx = 0
prev_frame = None
prev_detections = None  # will hold list of [x1,y1,x2,y2,conf] for pitch
prev_labels = None
prev_kps = None
buffer_frames = []
buffer_detections = []
buffer_labels = []
buffer_kps = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        # at EOF: if buffer exists, extrapolate using prev -> next (here next==prev)
        if buffer_frames and prev_detections is not None and prev_labels is not None:
            next_detections = prev_detections
            next_labels = prev_labels
            next_kps = prev_kps
            next_frame = prev_frame
            for i, f in enumerate(buffer_frames):
                frame_id = frame_idx - len(buffer_frames) + i
                alpha = (i + 1) / 5.0
                prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None,
                                                    0.5, 3, 15, 3, 5, 1.2, 0)
                pred_boxes = []
                pred_confs = []
                pred_labels = []
                pred_kps = []
                # propagate prev_detections using optical flow
                for box, label in zip(prev_detections, prev_labels):
                    x1, y1, x2, y2, conf = box
                    center_x = int(np.clip((x1 + x2) / 2, 0, width-1))
                    center_y = int(np.clip((y1 + y2) / 2, 0, height-1))
                    flow_x = float(flow[center_y, center_x, 0] * alpha)
                    flow_y = float(flow[center_y, center_x, 1] * alpha)
                    x1_pred = max(0, min(width, float(x1 + flow_x)))
                    y1_pred = max(0, min(height, float(y1 + flow_y)))
                    x2_pred = max(0, min(width, float(x2 + flow_x)))
                    y2_pred = max(0, min(height, float(y2 + flow_y)))
                    conf_pred = float(conf * (1 - alpha * 0.2))
                    if conf_pred > 0.1:
                        pred_boxes.append([x1_pred, y1_pred, x2_pred, y2_pred, conf_pred])
                        pred_confs.append(conf_pred)
                        pred_labels.append(label)
                # propagate keypoints if present
                if prev_kps:
                    for kp in prev_kps:
                        if kp[0] > 0 and kp[1] > 0:
                            cx = int(np.clip(kp[0], 0, width-1))
                            cy = int(np.clip(kp[1], 0, height-1))
                            kp_x = max(0, min(width, float(kp[0] + flow[cy, cx, 0] * alpha)))
                            kp_y = max(0, min(height, float(kp[1] + flow[cy, cx, 1] * alpha)))
                            pred_kps.append([kp_x, kp_y])
                        else:
                            pred_kps.append([float(kp[0]), float(kp[1])])
                # NMS
                pred_boxes, pred_confs, pred_labels = apply_nms(pred_boxes, pred_confs, pred_labels)
                frame_data = {
                    "frame_id": frame_id,
                    "pitch": {"boxes": [], "keypoints": pred_kps}
                    # removed "players" field per request
                }
                for box, conf, label in zip(pred_boxes, pred_confs, pred_labels):
                    box_data = {"bbox": box[:4], "conf": conf, "class": label}
                    if label == "pitch":
                        frame_data["pitch"]["boxes"].append(box_data)
                    else:
                        frame_data["pitch"]["boxes"].append(box_data)
                sparse_predictions.append(frame_data)
                f_drawn = draw_detections(f, pred_boxes, pred_labels, pred_kps)
                if save_output:
                    out.write(f_drawn)
        break

    if frame_idx % 5 == 0:
        # only run pitch model
        results_pitch = model_pitch(frame)[0]

        cur_boxes = []
        cur_labels = []
        cur_confs = []
        # collect pitch detections only
        for b in results_pitch.boxes:
            box = [float(x) for x in b.xyxy[0].cpu().numpy()]
            conf_val = float(b.conf[0])
            cur_boxes.append(box + [conf_val])
            cur_labels.append(results_pitch.names[int(b.cls[0])])
            cur_confs.append(conf_val)

        cur_kps = [[float(x), float(y)] for x, y in results_pitch.keypoints.xy[0].cpu().numpy()] if hasattr(results_pitch, "keypoints") and results_pitch.keypoints is not None and len(results_pitch.keypoints.xy) > 0 else []

        if prev_detections is not None and buffer_frames:
            next_detections = cur_boxes  # Sử dụng cur_boxes đã có conf
            next_labels = cur_labels
            next_kps = cur_kps
            next_frame = frame.copy()
            for i, f in enumerate(buffer_frames):
                frame_id = frame_idx - len(buffer_frames) + i
                alpha = (i + 1) / 5.0
                prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None,
                                                    0.5, 3, 15, 3, 5, 1.2, 0)
                pred_boxes = []
                pred_confs = []
                pred_labels = []
                pred_kps = []
                matches = match_boxes_by_iou(prev_detections, next_detections)
                used_next = set()
                for prev_idx, next_idx in matches:
                    # only match if class equal (mostly 'pitch' here)
                    if prev_labels[prev_idx] == next_labels[next_idx]:
                        box_prev = prev_detections[prev_idx]
                        box_next = next_detections[next_idx]
                        x1 = max(0, min(width, float(box_prev[0] + (box_next[0] - box_prev[0]) * alpha)))
                        y1 = max(0, min(height, float(box_prev[1] + (box_next[1] - box_prev[1]) * alpha)))
                        x2 = max(0, min(width, float(box_prev[2] + (box_next[2] - box_prev[2]) * alpha)))
                        y2 = max(0, min(height, float(box_prev[3] + (box_next[3] - box_prev[3]) * alpha)))
                        conf = float(box_prev[4] * (1 - alpha * 0.2))
                        if conf > 0.1:
                            pred_boxes.append([x1, y1, x2, y2, conf])
                            pred_confs.append(conf)
                            pred_labels.append(prev_labels[prev_idx])
                            used_next.add(next_idx)
                # unmatched prev -> propagate with flow
                for prev_idx, box in enumerate(prev_detections):
                    if prev_idx not in [m[0] for m in matches]:
                        center_x = int(np.clip((box[0] + box[2]) / 2, 0, width-1))
                        center_y = int(np.clip((box[1] + box[3]) / 2, 0, height-1))
                        flow_x = float(flow[center_y, center_x, 0] * alpha)
                        flow_y = float(flow[center_y, center_x, 1] * alpha)
                        x1_pred = max(0, min(width, float(box[0] + flow_x)))
                        y1_pred = max(0, min(height, float(box[1] + flow_y)))
                        x2_pred = max(0, min(width, float(box[2] + flow_x)))
                        y2_pred = max(0, min(height, float(box[3] + flow_y)))
                        conf_pred = float(box[4] * (1 - alpha * 0.2))
                        if conf_pred > 0.1:
                            pred_boxes.append([x1_pred, y1_pred, x2_pred, y2_pred, conf_pred])
                            pred_confs.append(conf_pred)
                            pred_labels.append(prev_labels[prev_idx])
                # unmatched next -> back-propagate (approx)
                for next_idx, box in enumerate(next_detections):
                    if next_idx not in used_next:
                        center_x = int(np.clip((box[0] + box[2]) / 2, 0, width-1))
                        center_y = int(np.clip((box[1] + box[3]) / 2, 0, height-1))
                        flow_x = float(flow[center_y, center_x, 0] * (1 - alpha))
                        flow_y = float(flow[center_y, center_x, 1] * (1 - alpha))
                        x1_pred = max(0, min(width, float(box[0] - flow_x)))
                        y1_pred = max(0, min(height, float(box[1] - flow_y)))
                        x2_pred = max(0, min(width, float(box[2] - flow_x)))
                        y2_pred = max(0, min(height, float(box[3] - flow_y)))
                        conf_pred = float(box[4] * (1 - alpha * 0.2))
                        if conf_pred > 0.1:
                            pred_boxes.append([x1_pred, y1_pred, x2_pred, y2_pred, conf_pred])
                            pred_confs.append(conf_pred)
                            pred_labels.append(next_labels[next_idx])
                # keypoints interpolation (pitch only)
                if prev_kps and next_kps:
                    for kp_prev, kp_next in zip(prev_kps, next_kps):
                        if kp_prev[0] > 0 and kp_prev[1] > 0 and kp_next[0] > 0 and kp_next[1] > 0:
                            kp_x = max(0, min(width, float(kp_prev[0] + (kp_next[0] - kp_prev[0]) * alpha)))
                            kp_y = max(0, min(height, float(kp_prev[1] + (kp_next[1] - kp_prev[1]) * alpha)))
                            pred_kps.append([kp_x, kp_y])
                        else:
                            pred_kps.append([float(kp_prev[0]), float(kp_prev[1])])
                pred_boxes, pred_confs, pred_labels = apply_nms(pred_boxes, pred_confs, pred_labels)
                frame_data = {
                    "frame_id": frame_id,
                    "pitch": {"boxes": [], "keypoints": pred_kps}
                    # removed "players" field per request
                }
                for box, conf, label in zip(pred_boxes, pred_confs, pred_labels):
                    box_data = {"bbox": box[:4], "conf": conf, "class": label}
                    if label == "pitch":
                        frame_data["pitch"]["boxes"].append(box_data)
                    else:
                        frame_data["pitch"]["boxes"].append(box_data)
                sparse_predictions.append(frame_data)
                f_drawn = draw_detections(f, pred_boxes, pred_labels, pred_kps)
                if save_output:
                    out.write(f_drawn)
            buffer_frames = []
            buffer_detections = []
            buffer_labels = []
            buffer_kps = []

        # now handle current predicted frame (every 5th)
        frame_data = {
            "frame_id": frame_idx,
            "pitch": {"boxes": [], "keypoints": cur_kps}
            # removed "players" field per request
        }
        for box, conf, label in zip(cur_boxes, cur_confs, cur_labels):
            box_data = {"bbox": box[:4], "conf": conf, "class": label}
            if label == "pitch":
                frame_data["pitch"]["boxes"].append(box_data)
            else:
                frame_data["pitch"]["boxes"].append(box_data)
        sparse_predictions.append(frame_data)

        final_frame = draw_detections(frame, cur_boxes, cur_labels, cur_kps)
        if save_output:
            out.write(final_frame)

        prev_detections = cur_boxes  # Cập nhật prev_detections với định dạng mới
        prev_labels = cur_labels
        prev_kps = cur_kps
        prev_frame = frame.copy()
    else:
        buffer_frames.append(frame.copy())
        buffer_detections.append(prev_detections)
        buffer_labels.append(prev_labels)
        buffer_kps.append(prev_kps)
    frame_idx += 1

cap.release()
if save_output:
    out.release()

with open("predictions_sparse.json", "w") as f:
    json.dump(sparse_predictions, f, indent=4)

# Hàm tính độ tương đồng
def calculate_iou(box1: list, box2: list) -> float:
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    inter_area = max(0, x2_i - x1_i) * max(0, y2_i - y1_i)
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0.0

def match_objects(boxes1: list, boxes2: list, iou_threshold: float = 0.7) -> list:
    matches = []
    used_j = set()
    for i, box1 in enumerate(boxes1):
        best_iou = 0.0
        best_j = -1
        for j, box2 in enumerate(boxes2):
            if j in used_j:
                continue
            iou = calculate_iou(box1['bbox'], box2['bbox'])
            if iou > best_iou and iou >= iou_threshold:
                best_iou = iou
                best_j = j
        if best_j != -1:
            matches.append((i, best_j))
            used_j.add(best_j)
    return matches

def calculate_keypoint_distance(kp1: list, kp2: list, img_width: int, img_height: int) -> float:
    distances = []
    for k in range(min(len(kp1), len(kp2))):
        if kp1[k][0] > 0 and kp1[k][1] > 0 and kp2[k][0] > 0 and kp2[k][1] > 0:
            dist = np.sqrt((kp1[k][0] - kp2[k][0])**2 + (kp1[k][1] - kp2[k][1])**2)
            distances.append(dist)
    if not distances:
        return 0.0
    avg_dist = np.mean(distances)
    normalized_dist = avg_dist / np.sqrt(img_width**2 + img_height**2)
    return normalized_dist

def calculate_frame_similarity(frame1: dict, frame2: dict, img_width: int, img_height: int, w1: float = 0.5, w2: float = 0.5) -> tuple:
     # Only use pitch boxes (players removed)
    boxes1 = frame1['pitch']['boxes']
    boxes2 = frame2['pitch']['boxes']
    matches = match_objects(boxes1, boxes2)
    num_matches = sum(1 for i, j in matches if boxes1[i]['class'] == boxes2[j]['class'])
    total_objects = len(boxes1) + len(boxes2) - len(matches)
    object_similarity = num_matches / total_objects if total_objects > 0 else 0.0
    conf_diff = []
    for i, j in matches:
        if boxes1[i]['class'] == boxes2[j]['class']:
            conf_diff.append(abs(boxes1[i]['conf'] - boxes2[j]['conf']))
    avg_conf_diff = np.mean(conf_diff) if conf_diff else 0.0
    keypoint_distances = []
    if frame1['pitch']['keypoints'] and frame2['pitch']['keypoints']:
        dist = calculate_keypoint_distance(frame1['pitch']['keypoints'], frame2['pitch']['keypoints'], img_width, img_height)
        keypoint_distances.append(dist)
    keypoint_similarity = 1.0 - np.mean(keypoint_distances) if keypoint_distances else 1.0
    frame_similarity = w1 * object_similarity + w2 * keypoint_similarity
    return frame_similarity, object_similarity, keypoint_similarity, avg_conf_diff

def calculate_video_similarity_from_json(json_path1: str, json_path2: str, img_width: int = 1920, img_height: int = 1080) -> dict:
    with open(json_path1, 'r') as f:
        predictions1 = json.load(f)
    with open(json_path2, 'r') as f:
        predictions2 = json.load(f)
    frame_similarities = []
    object_similarities = []
    keypoint_similarities = []
    conf_diffs = []
    for frame1 in predictions1:
        frame_id = frame1['frame_id']
        frame2 = next((f for f in predictions2 if f['frame_id'] == frame_id), None)
        if frame2 is None:
            continue
        frame_similarity, object_similarity, keypoint_similarity, avg_conf_diff = calculate_frame_similarity(
            frame1, frame2, img_width, img_height
        )
        frame_similarities.append(frame_similarity)
        object_similarities.append(object_similarity)
        keypoint_similarities.append(keypoint_similarity)
        conf_diffs.append(avg_conf_diff)
    video_similarity = np.mean(frame_similarities) if frame_similarities else 0.0
    avg_object_similarity = np.mean(object_similarities) if object_similarities else 0.0
    avg_keypoint_similarity = np.mean(keypoint_similarities) if keypoint_similarities else 0.0
    avg_conf_diff = np.mean(conf_diffs) if conf_diffs else 0.0
    total_frames = len(predictions1)
    predicted_frames = len([f for f in predictions2 if f['frame_id'] % 5 == 0])
    video_similarity_adjusted = video_similarity * (predicted_frames / total_frames) if total_frames > 0 else 0.0
    return {
        'video_similarity': video_similarity,
        'video_similarity_adjusted': video_similarity_adjusted,
        'avg_object_similarity': avg_object_similarity,
        'avg_keypoint_similarity': avg_keypoint_similarity,
        'avg_conf_diff': avg_conf_diff,
        'frame_similarities': frame_similarities,
        'frame_ids': [f['frame_id'] for f in predictions1 if any(f2['frame_id'] == f['frame_id'] for f2 in predictions2)]
    }

# Chạy hàm so sánh
json_path1 = "predictions_full.json"
json_path2 = "predictions_sparse.json"
results = calculate_video_similarity_from_json(json_path1, json_path2)

# In kết quả
print(f"Video Similarity (Average): {results['video_similarity']:.4f}")
print(f"Video Similarity (Adjusted for sparse prediction): {results['video_similarity_adjusted']:.4f}")
print(f"Average Object Similarity: {results['avg_object_similarity']:.4f}")
print(f"Average Keypoint Similarity: {results['avg_keypoint_similarity']:.4f}")
print(f"Average Confidence Difference: {results['avg_conf_diff']:.4f}")

# Lưu kết quả
with open("similarity_results.json", "w") as f:
    json.dump(results, f, indent=4)


0: 384x640 1 pitch, 12.2ms
Speed: 2.0ms preprocess, 12.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 12.3ms
Speed: 1.8ms preprocess, 12.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.4ms
Speed: 3.6ms preprocess, 10.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.8ms
Speed: 2.7ms preprocess, 8.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.4ms
Speed: 2.9ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.4ms
Speed: 1.8ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.9ms
Speed: 3.0ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x64